# Zadanie rekrutacyjne 4 - Wycena instrumentów

## Opis zadania

Twoim zadaniem jest obliczenie wartości bieżącej netto (NPV) i podstawowej miary ryzyka stóp procentowych (BPV) dla transakcji walutowego swapu stopy procentowej (CIRS) według stanu na dzień 17 lutego 2023.

Dane wejściowe do zadania zostały przekazane w formie plików CSV i obejmują:
- `transakcje.csv` - stylizowane parametry transakcji do wyceny,
- `krzywa_forward_EUR.csv` - stylizowana krzywa forward dla waluty EUR wyliczona na 17 lutego 2023,
- `krzywa_forward_PLN.csv` - stylizowana krzywa forward dla waluty PLN wyliczona na 17 lutego 2023,
- `krzywa_discount_EUR.csv` - stylizowana krzywa dyskontowa dla waluty EUR wyliczona na 17 lutego 2023,
- `krzywa_discount_PLN.csv` - stylizowana krzywa dyskontowa dla waluty PLN wyliczona na 17 lutego 2023,

Twoim zadaniem jest wykonanie obliczeń zgodnie z algorytmem opisanym poniżej. Jako wynik otrzymasz wartości NPV i BPV, osobne dla każdej nogi każdej transakcji. Zapoznaj się z opisem wszystkich obliczeń, a następnie przygotuj kod pozwalający rozwiązać zadanie.

Wykonując obliczenia przyjmij następujące konwencje długości roku:
- dla waluty PLN: ACT/ACT - do obliczeń przyjmuje się rzeczywisty czas pomiędzy wydarzeniami i rzeczywistą długość roku,
- dla waluty EUR: 30/360 - na cele obliczeń rok składa się z 12 miesięcy, z których każdy liczy 30 dni.

## Zależności

Zaimportuj pakiety, które będą Ci potrzebne w czasie rozwiązywania zadania.

In [140]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from datetime import datetime


## Import danych

Zaimportuj dane wejściowe z plików dostarczonych wraz z zadaniem. Opis zawartości każdego z pliku znajdziesz powyżej.

Krzywe wyceny mają formę:
|krzywa|tenor|data konstrukcji krzywej|data zapadalności|stopa zerokuponowa|czynnik dyskontowy|
|---|---|---|---|---|---|

Transakcje są dostarczone w formie tabeli:
|nr transakcji|data wyceny|strona transkcji|waluta|nominał|krzywa forwardowa|krzywa dyskontowa|daty płatności|okres odsetkowy|oprocentowanie w pierwszym okresie|data zapadalności|
|---|---|---|---|---|---|---|---|---|---|---|

Każda z transakcji obejmuje dwie nogi, każda w innej walucie i wyceniana w oparciu o zmienną stopę oprocentowania, odpowiadające stronie sprzedającej (S) i kupującej (B) transakcji. Zgodnie z konwencją, __dla strony kupującej przepływy pieniężne mają wartość ujemną, a dla strony sprzedającej - wartość dodatnią__.


Dla każdej transakcji, w kolumnie "daty płatności" znajduje się lista dni, w których kończą się poszczególne okresy odsetkowe transakcji. __Przyjmij, że data zakończenia okresu jest jednocześnie datą rozpoczęcia kolejnego__. Wraz z końcem ostatniego okresu odsetkowego, następuje również wypłata nominału transakcji. W wycenie instrumentu uwzględniane są jedynie przyszłe przepływy pieniężne, dlatego istotne są jedynie daty wypłat następujące po dacie wyceny.


Na podstawie tabeli transakcji __przygotuj harmonogram płatności__ zawierający daty rozpoczęcia i zakończenia poszczególnych okresów odsetkowych, który wykorzystasz w toku dalszych obliczeń.

In [141]:
try:
    transakcje = pd.read_csv('transactions.csv', sep=';')
    krzywa_forward_EUR = pd.read_csv('krzywa_forward_EUR.csv', sep=';')
    krzywa_forward_PLN = pd.read_csv('krzywa_forward_PLN.csv', sep=';')
    krzywa_discount_EUR = pd.read_csv('krzywa_discount_EUR.csv', sep=';')
    krzywa_discount_PLN = pd.read_csv('krzywa_discount_PLN.csv', sep=';')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

print("Transakcje columns:", transakcje.columns)
print("Krzywa Forward EUR columns:", krzywa_forward_EUR.columns)
print("Krzywa Forward PLN columns:", krzywa_forward_PLN.columns)
print("Krzywa Discount EUR columns:", krzywa_discount_EUR.columns)
print("Krzywa Discount PLN columns:", krzywa_discount_PLN.columns, '\n')

print("Transakcje:\n", transakcje.head())
print("Krzywa Forward EUR:\n", krzywa_forward_EUR.head())
print("Krzywa Forward PLN:\n", krzywa_forward_PLN.head())
print("Krzywa Discount EUR:\n", krzywa_discount_EUR.head())
print("Krzywa Discount PLN:\n", krzywa_discount_PLN.head())

Transakcje columns: Index(['nr transakcji', 'data wyceny', 'strona transakcji', 'waluta',
       'nominał', 'krzywa forwardowa', 'krzywa dyskontowa', 'daty płatności',
       'okres odsetkowy', 'oprocentowanie w pierwszym okresie',
       'data zapadalności', 'Unnamed: 11'],
      dtype='object')
Krzywa Forward EUR columns: Index(['krzywa', 'tenor', 'data konstrukcji krzywej', 'data zapadalności',
       'stopa zerokuponowa', 'czynnik dyskontowy'],
      dtype='object')
Krzywa Forward PLN columns: Index(['krzywa', 'tenor', 'data konstrukcji krzywej', 'data zapadalności',
       'stopa zerokuponowa', 'czynnik dyskontowy'],
      dtype='object')
Krzywa Discount EUR columns: Index(['krzywa', 'tenor', 'data konstrukcji krzywej', 'data zapadalności',
       'stopa zerokuponowa', 'czynnik dyskontowy'],
      dtype='object')
Krzywa Discount PLN columns: Index(['krzywa', 'tenor', 'data konstrukcji krzywej', 'data zapadalności',
       'stopa zerokuponowa', 'czynnik dyskontowy'],
      dtype='o

In [142]:
# Usuwanie cudzysłowów z dat
transakcje['data wyceny'] = transakcje['data wyceny'].str.replace("'", "")
transakcje['data zapadalności'] = transakcje['data zapadalności'].str.replace("'", "")

# Konwersja dat
transakcje['data wyceny'] = pd.to_datetime(transakcje['data wyceny'], format="%Y-%m-%d")
transakcje['data zapadalności'] = pd.to_datetime(transakcje['data zapadalności'], format="%Y-%m-%d")
transakcje['daty płatności'] = transakcje['daty płatności'].apply(eval)

# Konwersja dat w pozostałych plikach
for df in [krzywa_forward_EUR, krzywa_forward_PLN, krzywa_discount_EUR, krzywa_discount_PLN]:
    df['data konstrukcji krzywej'] = pd.to_datetime(df['data konstrukcji krzywej'], format="%d/%m/%Y")
    df['data zapadalności'] = pd.to_datetime(df['data zapadalności'], format="%d/%m/%Y")



## Obliczenie czynników dyskontowych

Czynnik dyskontowy $DF_{t_{0}, t_{1}}$ dla okresu pomiędzy datami $t_{0}$ a $t_{1}$ jest funkcją wykładniczą stopy zerokuponowej na ten sam okres ($ZC_{t_{0}, t_{1}}$) przeskalowanej przez frakcję roku YF, dla której jest liczony.
$$
DF_{t_{0}, t_{1}} = e^{-ZC_{t_{0}, t_{1}}\cdot YF} \tag{1}
$$ (label:1)

Frakcja roku to długość okresu $\Delta_{i}$ pomiędzy datami $t_{0}$ a $t_{1}$ ($\Delta_{t_{0}, t_{1}} = t_{1} - t_{0}$) podzielona przez przyjętą długość roku Y:
$$
YF_{t_{0}, t_{1}} = \frac{\Delta_{t_{0}, t_{1}}}{Y} \tag{2}
$$

Użyte w obliczeniach __wartości $\Delta_{i}$ oraz $Y$ będą zależały od konwencji dlugości roku obowiązującej dla danej waluty, określonej we wstępie do ćwiczenia__.

Krzywa dyskontowa zawiera wartości stopy zerokuponowej i obliczone czynniki dyskontowe dla wybranych dat (tenorów). Jeżeli tenor pokrywa się z datą wypłaty dla wybranej transakcji, wartości współczynników dyskontowych można odczytać bezpośrednio z krzywej. W pozostałych przypadkach, dla daty t wypadającej pomiędzy datami zapadalności $T_{0}$ i $T_{1}$ uwzględnionymi na krzywej wyceny, należy dokonać __interpolacji liniowej__ wartości stopy zerokuponowej:
$$
ZC_{T} = ZC_{T_{0}} + (ZC_{T_{1}} - ZC_{T_{0}}) \cdot \frac{t - T_{0}}{T_{1} - T_{0}} \tag{3}
$$

i podstawić otrzymaną wartość do równania (1), aby obliczyć czynnik dyskontowy.




Dla podanych transakcji, należy obliczyć wartości czynników dyskontowych dla dat podanych w kolumnie `daty płatności` w poniższy sposób:
1. Na odpowiedniej dla danej waluty __krzywej dyskontowej__ znajdź tenory, pomiędzy którymi wypada wybrana data płatności (lub z którymi się pokrywa).
2. Wyznacz stopę zerokuponową dla daty wypłaty przy użyciu równania (3).
3. Podstaw otrzymaną wartość do równania (1) podstawiając jako datę końca okresu wybraną datę płatności, a jako datę początku okresu - datę wyceny.

__Pamiętaj, że wartości stopy zeroprocentowej w tabeli są podane w %__.

In [143]:
def calculate_discount_factors(df_curve, payment_date, convention):
    dates = pd.to_datetime(df_curve['data zapadalności'], format='%d/%m/%Y').tolist()
    rates = df_curve['stopa zerokuponowa'].tolist()
    
    interp_function = interp1d(dates, rates, kind='linear', fill_value='extrapolate')
    rate_at_payment_date = float(interp_function(pd.Timestamp(payment_date)))
    
    if convention == 'ACT/ACT':
        year_fraction = (dates[dates.index(payment_date)] - dates[dates.index(payment_date) - 1]).days / 365.0
    elif convention == '30/360':
        year_fraction = 30 / 360.0
    
    discount_factor = np.exp(-rate_at_payment_date * year_fraction)
    
    return discount_factor

## Obliczenie stóp procentowych

Stopa procentowa jest określana wraz z rozpoczęciem każdego okresu odsetkowego. 
Dla biegnącego okresu, stopa procentowa jest podana dla każdej nogi transakcji w kolumnie `oprocentowanie w pierwszym okresie`.
Dla okresów przyszłych, należy ją wyznaczyć na podstawie krzywych forward na dzień rozpoczęcia okresu

Krzywa forward podaje wartości stóp zerokuponowych i czynników dyskontowych na konkretne daty. Pomiędzy tymi datami, wartości należy wyznaczyć przez interpolację. 
Do wyceny instrumentów, wykorzystaj __interpolację liniową__, zgodnie z równaniami (1) i (3). Aby obliczyć stopy oprocentowania dla przyszłych okresów odsetkowych, wykonaj ponoiższe obliczenia:

1. Dla pierwszego (biegnącego) okresu odsetkowego, stopa oprocentowania jest znana i podana w kolumnie `oprocentowanie w pierwszym okresie` w danych wejściowych.
Dla pozostałych okresów:
2. Zgodnie z metodologią przestawioną w poprzednim punkcie, na podstawie odpowiedniej dla danej waluty __krzywej forwardowej__ wyznacz czynniki dyskontowe $DF_{start}$ dla okresu pomiędzy __datą wyceny__ a __datą rozpoczęcia__ każdego okresu odsetkowego. __Nie możesz tego wykonać dla biegnącego okresu odsetkowego, którego data rozpoczęcia wypada przed datą wyceny (bądącą równocześnie datą konstrukcji krzywej wyceny)__.
3. Analogicznie, na podstawie __krzywej forwardowej__ wyznacz czynniki dyskontowe $DF_{end}$ dla okresu pomiędzy __datą wyceny__ a __datą zakończenia__ każdego okresu odsetkowego.
4. Dla każdego okresu odsetkowego zaczynającego się w $t_{start}$ i kończącego w $t_{end}$, oblicz interpolowaną stopę oprocentowania $r_{t_{start}, t_{end}}$ na podstawie wyznaczonych współczynników dyskontowych:
$$
r_{t_{start}, t_{end}} = (\frac{DF_{start}}{DF_{end}} - 1) \cdot YF_{t_{start}, t_{end}} \cdot 100\% \tag{4}
$$

__Pamiętaj, że wartości stopy zeroprocentowej w tabeli są podane w %__.





In [144]:
def calculate_interest_rates(transactions_df, fwd_curve_df):
    interest_rates = []
    for index, row in transactions_df.iterrows():
        start_date = row['data wyceny']
        payment_dates = row['daty płatności'].split(',')
        payment_dates = [date.strip("'") for date in payment_dates]
        payment_dates = [datetime.strptime(date, '%Y-%m-%d') for date in payment_dates]
        
        fwd_rates = []
        for i in range(len(payment_dates)):
            end_date_str = payment_dates[i].strftime('%d/%m/%Y')
            fwd_rate = fwd_curve_df.loc[(fwd_curve_df['data konstrukcji krzywej'] == start_date) & 
                                        (fwd_curve_df['data zapadalności'] == end_date_str), 'stopa zerokuponowa']
            if not fwd_rate.empty:
                rate_start = float(fwd_rate.iloc[0]) / 100.0
                fwd_rates.append(rate_start)
        
        interest_rates.append(fwd_rates)
    
    return interest_rates

## Obliczenie przepływów pieniężnych

Dla każdego okresu odsetkowego zaczynającego się w $t_{0}$ i kończącego w $t_{1}$, przepływ pieniężny CF jest równy nominałowi nogi transakcji przemnożonemu przez stopę procentową dla danego okresu i frakcję roku odpowiadającą jego długości. 

Kierunek przepływu (znak otrzymanej wartości) zależy od strony transakcji. Jest to uwzględnione przez współczynnik $s_{B/S}$.
$$
CF_{t{0}, t_{1}} = s_{B/S} \cdot N \cdot r_{t_{0}, t_{1}} \cdot YF_{t_{0}, t_{1}} \tag{5}
$$.

Wykonaj obliczenia przepływów pieniężnych przyjmując, że nominał nie ulega amortyzacji - jego wartość jest taka sama przez cały okres trwania umowy.

In [145]:
def calculate_cash_flows(transactions_df, discount_curve_df, interest_rates):
    cash_flows = []
    
    for index, row in transactions_df.iterrows():
        nominal = row['nominał']
        side = -1 if row['strona transakcji'] == 'B' else 1
        
        payment_dates = row['daty płatności'].split(',')
        payment_dates = [date.strip() for date in payment_dates]
        
        for i in range(len(payment_dates) - 1):
            start_date = datetime.strptime(payment_dates[i], '%Y-%m-%d')
            end_date = datetime.strptime(payment_dates[i + 1], '%Y-%m-%d')
            
            year_fraction = (end_date - start_date).days / 365.0
            
            #discount_factor_start = calculate_discount_factors(discount_curve_df, start_date, row['krzywa dyskontowa'])
            #discount_factor_end = calculate_discount_factors(discount_curve_df, end_date, row['krzywa dyskontowa'])
            
            cash_flow = side * nominal * interest_rates[i] * year_fraction
            cash_flows.append(cash_flow)
    
    return cash_flows


## Obliczenie NPV

Wartość bieżąda netto (NPV) transakcji CIRS na dzień wyceny ($t_{pricing}$) to suma przepływów pieniężnych pomnożonych przez czynnik dyskontowy obliczony na dzień ich wypłaty ($DF_{T_{CF}}$). Dodatkowo, w raz z wypłatą ostatniego przepływu odsetkowego w dniu zakończenia transakcji ($t_{last}$), następuje wypłata nominału $N$, którego zdyskontowana (na tę datę) wartość jest uwzględniona w wycenie. Ostatecznie, wzór na NPV przyjmuje postać:

$$
NPV = s_{B/S} \cdot N \cdot DF_{t_{pricing}, t_{last}} + \sum_{t_{CF, 1}}^{t_{last}}  CF_{t_{CF}} \cdot DF_{t_{pricing}, t_{CF}} \tag{6}
$$

Wykonaj odpowiednie obliczenia dla każdej z nóg obydwu transakcji.

In [146]:
def calculate_npv(transactions_df, discount_curve_df, interest_rates):
    npvs = []
    
    for index, row in transactions_df.iterrows():
        payment_dates = row['daty płatności'].split(',')
        payment_dates = [date.strip() for date in payment_dates]
        
        npv = 0
        
        for i in range(len(payment_dates) - 1):
            start_date = datetime.strptime(payment_dates[i], '%Y-%m-%d')
            end_date = datetime.strptime(payment_dates[i + 1], '%Y-%m-%d')
            
            discount_factor_end = calculate_discount_factors(discount_curve_df, end_date, row['krzywa dyskontowa'])
            
            side = -1 if row['strona transakcji'] == 'B' else 1
            nominal = row['nominał']
            year_fraction = (end_date - start_date).days / 365.0
            cash_flow = side * nominal * interest_rates[i] * year_fraction
            
            npv += cash_flow * discount_factor_end
        
        last_end_date = datetime.strptime(payment_dates[-1], '%Y-%m-%d')
        discount_factor_last = calculate_discount_factors(discount_curve_df, last_end_date, row['krzywa dyskontowa'])
        npv += side * nominal * discount_factor_last
        
        npvs.append(npv)
    
    return npvs

## Obliczenie BPV

Aby obliczyć podstawową miarę ryzyka stóp procentowych dla transakcji (BPV), przygotuj przesunięte krzywe wyceny __podnosząc wartości stopy zerokuponowej na każdej krzywej dyskontowej i forwardowej o 1 punkt bazowy ($1 bp = 0.01\%$)__. Na podstawie tak wyznaczonej stopy zerokuponowej, przelicz dla krzywej __współczyynnik dyskontowy zgodnie z równaniem (1)__. Pozostawiająć __pozostałe wartości niezmienione, powtórz wszystkie kroki oblieczeń opisane powyżej z użyciem przesuniętych krzywych__.  Otrzymasz w ten sposób wartość $NPV_{+1bp}$.

Miarę ryzyka BPV obliczysz, odejmując od otrzymanej wartości wcześniej obliczone NPV transakcji.

$$
BPV = NPV_{+1bp}-NPV \tag{7}
$$

Wykonaj odpowiednie obliczenia.

In [147]:
def calculate_bpv(transactions_df, discount_curve_df, fwd_curve_df):
    bpvs = []
    
    for index, row in transactions_df.iterrows():
        nominal = row['nominał']
        side = -1 if row['strona transakcji'] == 'B' else 1
        
        payment_dates = row['daty płatności'].split(',')
        payment_dates = [date.strip() for date in payment_dates]
        
        for i in range(len(payment_dates) - 1):
            start_date = datetime.strptime(payment_dates[i], '%Y-%m-%d')
            end_date = datetime.strptime(payment_dates[i + 1], '%Y-%m-%d')
            
            year_fraction = (end_date - start_date).days / 365.0
            
           # discount_factor_start = calculate_discount_factors(discount_curve_df, start_date, row['krzywa dyskontowa'])
            #discount_factor_end = calculate_discount_factors(discount_curve_df, end_date, row['krzywa dyskontowa'])
            
            start_date_str = start_date.strftime('%d/%m/%Y')
            end_date_str = end_date.strftime('%d/%m/%Y')
            
            fwd_rates = fwd_curve_df.loc[(fwd_curve_df['data konstrukcji krzywej'] == start_date_str) & 
                                         (fwd_curve_df['data zapadalności'] == end_date_str), 'stopa zerokuponowa']
            
            rate_start = float(fwd_rates.iloc[0])
            
            fwd_rates_next = fwd_curve_df.loc[(fwd_curve_df['data konstrukcji krzywej'] == start_date_str) & 
                                              (fwd_curve_df['data zapadalności'] == payment_dates[i].strftime('%d/%m/%Y')), 'stopa zerokuponowa']
            
            rate_end = float(fwd_rates_next.iloc[0])
            
            bpv = side * nominal * year_fraction * ((rate_end - rate_start) / 100)
            bpvs.append(bpv)
    
    return bpvs

## Wyeksportuj wyniki

Jako wynik zadania, wyeksportuj do plików CSV:
- wyniki obliczeń w formie tabeli zawierającą zestawione wartości NPV i BPV dla każdej nogi każdej z transakcji,
- harmonogramy dla każdej nogi każdej transakcji, zawierające:
    - datę rozpoczęcia okresu odsetkowego,
    - datę zakończenia okresu odsetkowego,
    - stopę oprocentowania dla danego okresu,
    - czynnik dyskontowy na koniec okresu,
    - przepływ pieniężny dla danego okresu.

W pakiecie podsumowującym zadanie, prześlij:
- otrzymane dane wejściowe,
- plik Jupyter Notebook zawierający kod rozwiązania zadania oraz wyniki obliczeń,
- wyeksportowane pliki wynikowe.

__Pamiętaj, aby skonstruować tabele i nazwać pliki tak, aby było jasne, której transakcji dotyczy.__

In [150]:
def calculate_values(transactions_df, discount_curve_EUR_df, discount_curve_PLN_df, fwd_curve_EUR_df, fwd_curve_PLN_df):
    interest_rates_EUR = calculate_interest_rates(transactions_df[transactions_df['waluta'] == 'EUR'], fwd_curve_EUR_df)
    interest_rates_PLN = calculate_interest_rates(transactions_df[transactions_df['waluta'] == 'PLN'], fwd_curve_PLN_df)
    npvs_EUR = calculate_npv(transactions_df[transactions_df['waluta'] == 'EUR'], discount_curve_EUR_df, interest_rates_EUR)
    npvs_PLN = calculate_npv(transactions_df[transactions_df['waluta'] == 'PLN'], discount_curve_PLN_df, interest_rates_PLN)
    result_df = pd.concat(transactions_df[['ID_transakcji', 'waluta', 'NPV', 'BPV']], pd.DataFrame({'NPV': npvs_EUR, 'BPV': npvs_PLN}), axis=1).to_csv('all_results.csv', index=False)
    return result_df

In [151]:
result_df = calculate_values(transakcje, krzywa_discount_EUR, krzywa_discount_PLN,krzywa_forward_EUR, krzywa_forward_PLN)

#result_df[['ID_transakcji', 'waluta', 'NPV', 'BPV']].to_csv('npv_bpv_results.csv', index=False)
result_df.to_csv('all_results.csv', index=False)


AttributeError: 'list' object has no attribute 'split'